In [1]:
#reputation_api

# Reputation Service API, including Rating Service and Ranking Service

import abc

#TODO @anton provide proper parameters for the methods


"""
Reputation Generic Service interface definition
"""        
class ReputationService(abc.ABC):

	"""
	Input: dict of all parameters that needs to be set (not listed parameters are not affected)
	Output: 0 on success, integer error code on error 
	"""
	@abc.abstractmethod
	def set_parameters(self,parameters):
		pass

	@abc.abstractmethod
	def get_parameters(self):
		pass


"""
Reputation Rating Service interface definition
"""        
class RatingService(ReputationService):

	"""
	Input: List of dicts with the key-value pairs for the attributes: "from","type","to","value","weight","time"
	Output: 0 on success, integer error code on error 
	"""
	@abc.abstractmethod
	def put_ratings(self,ratings):
		pass

	"""
	Input: filter as dict of the following:
		since - starting time inclusively
		until - ending time inclusively
		ids - list of ids to retrieve incoming AND outgoing ratings BOTH
		from - list of ids to retrieve outgoing ratings ONLY (TODO later)
		to - list of ids to retrieve incoming ratings ONLY (TODO later)
	Output: tuple of the pair:
		0 on success, integer error code on error
		List of dicts with the key-value pairs for the attributes: "from","type","to","value","weight","time"
	"""
	@abc.abstractmethod
	def get_ratings(self,filter):
		pass

	"""
	Input: None
	Output: 0 on success, integer error code on error 
	"""
	@abc.abstractmethod
	def clear_ratings(self):
		pass
		
"""
Reputation Ranking Service interface definition
"""        
class RankingService(ReputationService):

	"""
	Input: Date to update the ranks for
	Output: 0 on success, integer error code on error 
	"""
	@abc.abstractmethod
	def update_ranks(self,date):
		pass

	"""
	Input: Date and list of dicts with two key-value pairs for "id" and "rank" 
	Output: 0 on success, integer error code on error 
	"""
	@abc.abstractmethod
	def put_ranks(self,date,ranks):
		pass

	"""
	Input: filter as dict of the following:
		date - date to provide the ranks
		ids - list of ids to retrieve the ranks
	Output: tuple of the pair:
		0 on success, integer error code on error
		List of dicts with the two key-value pairs for "id" and "rank"
	"""
	@abc.abstractmethod
	def get_ranks(self,filter):
		pass

	"""
	Input: None
	Output: 0 on success, integer error code on error 
	"""
	@abc.abstractmethod
	def clear_ranks(self):
		pass

	@abc.abstractmethod
	def get_ranks_dict(self,filter):
		pass

In [2]:
#reputation service base


"""
Abstract Reputation Service wrapper around
"""        

import os
import subprocess


class ReputationServiceBase(RatingService,RankingService):

	def __init__(self, name, verbose=False):
		self.name = name #service parameter, no impact on algorithm, name of the storage scheme
		self.verbose = verbose #service parameter, no impact on algorithm, impact on log level 
		self.parameters = {}
		self.parameters['default'] = 0.5 # default (initial) reputation rank
		self.parameters['decayed'] = 0.0 # decaying (final) reputaion rank, may be equal to default one
		self.parameters['conservatism'] = 0.5 # blending factor between previous (default) rank and differential one 
		self.parameters['precision'] = 0.01 # Used to dound/up or round down financaial values or weights as value = round(value/precision)
		self.parameters['weighting'] = True # forces to weight ratings with financial values, if present
		self.parameters['denomination'] = False # forces to denominate weighted ratings with sum of weights
		self.parameters['fullnorm'] = True # full-scale normalization of incremental ratings
		self.parameters['liquid'] = True # forces to account for rank of rater
		self.parameters['logranks'] = True # applies log10 to ranks
		self.parameters['logratings'] = True # applies log10(1+value) to financial values and weights
		self.parameters['downrating'] = False # boolean option with True value to translate original explicit rating values in range 0.5-0.0 to negative values in range 0.0 to -1.0 and original values in range 1.0-0.5 to interval 1.0-0.0, respectively
		self.parameters['update_period'] = 1 # number of days to update reputation state, considered as observation period for computing incremental reputations
		self.parameters['aggregation'] = False #TODO support in Aigents, aggregated with weighted average of ratings across the same period
		self.parameters['unrated'] = False # whether to store default ranks of unrated agents and let them decay 
		self.parameters['ratings'] = 1.0 # to which extent account contribution of explicit and implicit ratings to reputation
		self.parameters['spendings'] = 0.0 # to which extent account contribution of spendings ("prrof-of-burn") to reputation
		self.parameters['parents'] = 0.0 # to which extent reputation of the "child" (product) is affected by the reputation of the "parent" (vendor)
		self.parameters['predictiveness'] = 0.0 # to which extent account rank is based on consensus between social consensus and ratings provided by the account
		self.parameters['rating_bias'] = False # whether to weigth ratings based on pessimism of the prior ratings
		

	"""
	Utility wrapper around get_ranks, returns None in case of error, so ranks  = get_ranks_dict(...) should be checked for None, and if it is None, the get_ranks(...) may be used to decipher the error code.
	Input: filter as dict of the following:
		date - date to provide the ranks
		ids - list of ids to retrieve the ranks
	Output:
		dictionary of key-value pairs with reputation "ranks" by "id" on success, None on error
	"""
	def get_ranks_dict(self,filter):
		ranks_dict = {}
		res, ranks = self.get_ranks(filter)
		if res != 0:
			return None
		for rank in ranks:
			ranks_dict[rank['id']] = rank['rank']
		return ranks_dict
	
	"""
	TODO: implement in derived imlementations
	"""
	def set_parent(self,parent_id,list_of_children_ids):
		return 0

In [3]:
# Reputation_calculation

### Import packages
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from logging import debug
from math import ceil, floor

### Get strings between two strings; will be useful when extracting the date.
def find_between( s, first, last ):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""
### Get strings between two strings; will be useful when extracting the date. Different way of implementing the
### same thing as above.
# def find_between_r( s, first, last ):
#     try:
#         start = s.rindex( first ) + len( first )
#         end = s.rindex( last, start )
#         return s[start:end]
#     except ValueError:
#         return ""

### Below two functions will make sure we get difference between times.
def parse_prefix(line, fmt):
    cover = len(datetime.datetime.now().strftime(fmt))
    return datetime.strptime(line[:cover], fmt)
### Calculate days between d2 and d1.
def days_between(d1, d2):
    return abs((d2 - d1).days) 

def downratings(condition,ratings):
    if condition:
    	### it is expected current_max to be 1.
        current_max = 1### Note that maximum is set to 1 as per documentation. We do not allow 
        ### values higher than 1.
        i = 0
        while i<len(ratings):
            if ratings[i]['value']>current_max:
                ### as soon as we find 1 value above 1, we raise an error.
                raise ValueError("Downratings are not on the scale of 0 to 1, as required.")
            i+=1
        
        i=0
        while i<len(ratings):
        	### it is expected ratings to be converted to range -100 to +100 (or -1.0 to +1.0 on -100% to +100% scale)
            ratings[i]['value'] = ratings[i]['value']/current_max
            ### current_max is set to 1, so we are essentially dividing by 1. Now, below, we are converting everything
            ### to the range of -1 to 1.
            if ratings[i]['value']<0.25:
                ratings[i]['value'] = ratings[i]['value']/0.25-1
            else:
                ratings[i]['value'] = (ratings[i]['value']-0.25)/0.75
            ### Then we multiply by 100, so we get it btw -100 and 100.    
            ratings[i]['value'] = ratings[i]['value'] * 100
            i+=1
        return(ratings)
    else:
        ### If downratings=false, then we do nothing.
        return(ratings)
### Rounding is fixed here. Normal rounding in Python rounds 0.5 to 0, this little function prevents that.    
def my_round(n, ndigits):
    part = n * 10 ** ndigits
    delta = part - int(part)
    # always round "away from 0"
    if delta >= 0.5 or -0.5 < delta <= 0:
        part = ceil(part)
    else:
        part = floor(part)
    return part / (10 ** ndigits)

### Transforming ratings to logarithm, if needed. logratings might or might not be set to true.
def transform_ratings(ratings, logratings):
    if logratings:
        i=0        
        while i<len(ratings):
            ### Transformations of weight depending on the value. If smaller than 0, then we need to adjust a bit.
            if ratings[i]['weight']!=None:
                if ratings[i]['weight']<0:
                    ratings[i]['weight'] = -np.log10(1-ratings[i]['weight'])
                else:
                    ratings[i]['weight'] = np.log10(1+ratings[i]['weight'])
            else:
                ### We do the same with value.
                if ratings[i]['value']<0:
                    ratings[i]['value'] = -np.log10(1-ratings[i]['value'])
                else:
                    ratings[i]['value'] = np.log10(1+ratings[i]['value'])#np.log10(1+ratings[i]['value'])
            i+=1
    return(ratings)
### Weight calculation. Only problem is if we have no value number. If we do, we just call logratings_precision.
def weight_calc(value,lograting,precision,weighting):
    if value != None:
        return(logratings_precision(value,lograting,precision,weighting))
    else:
        return(1,None)
###   Get starting dates and first occurances of each addresses. Also, preparation or arrays and other data
### to be used in the future.
### Note; Given that we take an approach where we don't need first_occurance, we decide to put as a default option
### need_occurance=False.
def reputation_calc_p1(new_subset,conservatism,precision,temporal_aggregation=False,need_occurance=False,
                       logratings=False,downrating=False,weighting=True,rater_bias = None,averages = None):
    ### need_occurance is set to false by default and might even be removed for good. It was made in order to
    ### facilitate some other approaches towards updating rankings, which we decided not to use in the end.
    #### We will need from, to, amount, the rest is not necessary to have - let's save memory.
    ### Now we will just store the first occurance of each account in a dictionary.
    ##  Inputs are dictionaries, arrays and True/False statements.
    ### We change the subeset that is incoming in order to put downratings transformation.
    new_subset = downratings(downrating,new_subset)
    if rater_bias != None:
        rater_bias,average_rating = update_biases(rater_bias,new_subset,conservatism)
        
        our_averages = dict()
        for k in averages:
            for j in averages[k].keys():
                if j in our_averages.keys():
                    our_averages[j].append(averages[k][j])
                else:
                    our_averages[j] = [averages[k][j]]
        our_average = dict()
        for k in our_averages.keys():
            our_average[k] = np.mean(our_averages[k])
        new_subset = fix_rater_bias(new_subset,rater_bias,our_average)    
            
    i=0
    new_array = []
    israting = True
    while i<len(new_subset):
        if 'value' in list(new_subset[i].keys()):
            ### put ratings in array. Note, that we don't always have information about rating, that is
            ### what ratings were given by specific customers.
            ### This array is standardized.
            if 'weight' in new_subset[i].keys():
                new_array.append([new_subset[i]['from'],new_subset[i]['to'],new_subset[i]['weight'],new_subset[i]['value']])
            else:
                new_array.append([new_subset[i]['from'],new_subset[i]['to'],None,new_subset[i]['value']])
        else:
            israting = False
            if 'weight' in new_subset[i].keys():
                new_array.append([new_subset[i]['from'],new_subset[i]['to'],new_subset[i]['weight']])
            else:
                new_array.append([new_subset[i]['from'],new_subset[i]['to'],None])
        i+=1
    ### We make array of dates and transactions to specific agents.
    dates_array = []
    to_array = []
    i = 0
    ### we have array of only dates and array of ids which are getting transactions.
    while i<len(new_subset):
        dates_array.append(new_subset[i]['time'])
        to_array.append(new_subset[i]['to'])
        i+=1
    ### In case we have temporal aggregation
    if temporal_aggregation:
        from_data = []
        to_data = to_array
        i = 0
        while i<len(new_array):
            ### we merge all the 'from' data.
            from_data.append(int(new_array[i][0]))
            i+=1
            
        ### Temporal aggregation=True;
        ### First let's just find all the duplicates;
        ### We merge from and to arrays and look for unique ones...
        ### The idea of this code is that if there were multiple transactions in a day, we merge them and look at
        ### the averages.
        merged = []
        i=0
        while i<len(from_data):
            ### We get all the from-to id combinations.
            newnr = str(from_data[i])+"_"+str(to_data[i])
            merged.append(newnr)
            i+=1
        ### Here we just count how many times it appears
        already_used = {}
        ### We count how many times each combination appeared.
        for i in merged:
            if i in already_used.keys():
                already_used[i] = already_used[i] + 1
            else:
                already_used[i] = 1
        ### Good, now we know exact nr of transactions for each pair...    
        
        #### merged data has the same indexing as new_array. 
        i = 0
        ### If exists, pass, otherwise this:
        ### We sum up each feature.
        already_used2 = {}
        new_array2 = []
        to_array2 = []
        amounts = {}
        ratings = {}
        while i<len(merged):
            if merged[i] in already_used2.keys():
                new_rating, new_weight = weight_calc(new_array[i],logratings,precision,weighting)
                amounts[merged[i]] = amounts[merged[i]] + new_rating
                if israting:
                    ### then we sum up ratings.
                    ratings[merged[i]] = ratings[merged[i]] + new_array[i][3]               
            else:
                already_used2[merged[i]]=1
                new_rating, new_weight = weight_calc(new_array[i],logratings,precision,weighting)
                amounts[merged[i]] = new_rating
                if israting:
                    ratings[merged[i]] = new_array[i][3]                        
            i+=1
        i=0
        ### And divide it by the number of times it appears - getting average.
        already_used2 = {}
        while i<len(merged):
            if merged[i] in already_used2.keys():
                pass
            else:
                already_used2[merged[i]]=1
                ### Just set some value.
                new_array2.append(new_array[i])
                new_array2[len(new_array2)-1][2] = amounts[merged[i]]/already_used[merged[i]]
                if israting:
                    new_array2[len(new_array2)-1][3] = ratings[merged[i]]/already_used[merged[i]]
                
                to_array2.append(to_array[i])
            i+=1
        new_array = new_array2
        to_array = to_array2
    if rater_bias != None:
        return(new_array,dates_array,to_array,rater_bias,average_rating)
    else:
        return(new_array,dates_array,to_array,rater_bias)
### Get new reputations in case we do not yet have the old ones.
def update_reputation(reputation,new_array,default_reputation,spendings):
    i = 0
    new_ids = []
    while i<len(new_array):
        ### If we already have it, we do nothing in this function...
        ### The rest is also checking for "to" transactions and doing the same thing there..
        if new_array[i][1] in reputation:
            ### If reputation already has an id, we go on, otherwise we add default reputation.
            pass
        else:
            new_ids.append(new_array[i][1])
            reputation[new_array[i][1]] = default_reputation
        ### If we have spendings, we need reputation also for buyers. We make it default if it does not exist yet.    
        if spendings>0:
            if new_array[i][0] in reputation:
                pass
            else:
                new_ids.append(new_array[i][0])
                reputation[new_array[i][0]] = default_reputation
            
        i+=1      
    return(reputation)


### Logratings calculation, calculating weights and fixing for precision.
def logratings_precision(rating,lograting,precision,weighting):
    new_weight = None # assume no weight computed by default
    ### if weighting = False, then we return values only.
    if not weighting:
        return(rating[3],None)
    if lograting:
        ### We go through few posibilities about what can happen.
        ### If there are no weights then:
        if rating[2] == None:
            ### depending on precision, we make a log transformation with or without it.
            if precision==None:
                new_rating = np.log10(1+ rating[3])
            else:
                new_rating = np.log10(1+ int(rating[3]/precision))
        else:
            ### if we have no values;
            if rating[3] == None:
                ### Then we work with weights only.
                if precision==None:
                    new_rating = np.log10(1+ rating[2])
                else:
                    new_rating = np.log10(1+ int(rating[2]/precision))
            else:
                ### if we have values and weights, we multiply them together.
                if precision==None:
                    new_weight = np.log10(1+ rating[2])
                else:
                    new_weight = np.log10(1+ rating[2]/precision)
                new_rating = my_round(new_weight * rating[3],0)
    else:
        ### If not lograting, we do not do log transformation.
        if precision==None:
            precision=1
        if rating[2] == None:
            new_rating = rating[3]/precision
        else:
            if rating[3] == None:
                new_rating = rating[2]/precision
            else:
                new_weight = rating[2]/precision
                new_rating = rating[3] * new_weight
    new_rating = my_round(new_rating,0) 
    return(new_rating,new_weight) #return weighted value Fij*Qij to sum and weight Qij to denominate later in dRit = Î£j (Fij * Qij * Rjt-1 ) / Î£j (Qij)

def update_biases(previous_bias,new_arrays, conservatism):
    all_rating = dict()
    i = 0
    while i<len(new_arrays):
        if new_arrays[i]['from'] in all_rating.keys():
            all_rating[new_arrays[i]['from']].append(new_arrays[i]['value'])
        else:
            all_rating[new_arrays[i]['from']] = [new_arrays[i]['value']]
        i+=1
    averages = dict()
    for k in all_rating.keys():
        averages[k] = np.mean(all_rating[k])
    unique_ids = []
    for k in averages.keys():
        if k in unique_ids:
            pass
        else:
            unique_ids.append(k)
    for k in previous_bias.keys():
        if k in unique_ids:
            pass
        else:
            unique_ids.append(k)
    for k in averages.keys():
        if k in unique_ids:
            pass
        else:
            unique_ids.append(k)
        
        
    new_bias = dict()
    for k in unique_ids:
        if k in averages.keys() and k in previous_bias.keys():
            new_bias[k] = averages[k] * (1-conservatism) + conservatism * previous_bias[k]
        else:
            if k in averages.keys():
                new_bias[k] = averages[k] * (1-conservatism) + conservatism ### This is how we are supposed to 
                ### treat first customer based on the https://docs.google.com/document/d/1-O7avb_zJKvCXRvD0FmvyoVZdSmkDMlXRB5wsavpSAM/edit#
            if k in previous_bias.keys():
                new_bias[k] = previous_bias[k]
                
    return(new_bias,averages)


def fix_rater_bias(new_array,biases,average):
    i = 0
    while i<len(new_array):
        if new_array[i]['from'] in average.keys():
            new_array[i]['value'] = new_array[i]['value'] * (1-my_round(average[new_array[i]['from']],0))
        else:
            new_array[i]['value'] = new_array[i]['value']
        
        i+=1
    return (new_array)

### Get updated reputations, new calculations of them...
### We calculate differential here.
def calculate_new_reputation(logging,new_array,to_array,reputation,rating,precision,previous_rep,default,unrated,normalizedRanks=True,weighting=True,denomination=True,liquid = False,logratings=False,logranks=True,predictiveness = 0,predictive_data = dict()):
    ### The output will be mys; this is the rating for that specific day (or time period).
    ### This is needed; first create records for each id. mys is a differential.
    mys = {}
    myd = {} # denominators 
    i = 0
    while i<len(new_array):
        if new_array[i][1] in mys:
            pass
        else:
            ### We first set all differential ids to 0.
            mys[new_array[i][1]] = 0
        i+=1
    ## getting the formula for mys.
    unique_ids = np.unique(to_array)
    k=0
    i = 0
    to_array = np.array(to_array)
    ### Formula differs based on conditions. If ratings are included, formula includes ratings, then there are weights, etc.
    prev_rep1 = dict()
    prev_rep1a = dict()
    while i<len(unique_ids):
        amounts = []
        denominators = []
        ### Here we get log transformation of each amount value. 
        get_subset = np.where(to_array==unique_ids[i])[0]
        for k in get_subset:
            if weighting:
                ### Calculate ratings and weights.
                new_rating, new_weight = weight_calc(new_array[k],logratings,precision,weighting)
                ### Then we multiply this with rater's current reputation. Few options are taken into account, such as
                ### if it is liquid reputation, then we set it to 1...
                my_rater_rep, prev_rep1 = rater_reputation(reputation,new_array[k][0],default,previous_rep,liquid,new_array[k][1],predictiveness,predictive_data)
                for k in prev_rep1.keys():
                    prev_rep1a[k] = prev_rep1[k]
                amounts.append(new_rating * my_rater_rep)
                text = "from: " + str(new_array[i][0]) + ", to: " + str(new_array[i][1]) + ", value: " + str(new_array[i][2]) + ", weight: " + str(new_array[i][3])," calculated rating: ",new_rating
                logging.debug(text)
                ### if we have weights and denomination, then we append some denominators.
                if denomination and new_weight is not None:
                	denominators.append(new_weight) # denomination by sum of weights in such case
            else:
                new_rating, new_weight = weight_calc(new_array[k],logratings,precision,weighting)
                new_rating = my_round(new_rating,0)
                my_rater_rep, prev_rep1 = rater_reputation(reputation,new_array[k][0],default,previous_rep,liquid,new_array[k][1],predictiveness,predictive_data)
                for k in prev_rep1.keys():
                    prev_rep1a[k] = prev_rep1[k]
                amounts.append(new_rating * my_rater_rep)
                text = "from: " + new_array[i][0] + ", to: " + str(new_array[i][1]) + ", value: " + str(new_array[i][2]) + ", weight: " + str(new_array[i][3])," calculated rating: ",str(new_rating)
                logging.debug(text)
                #no need for denomination by sum of weights in such case 
        ### After we are done collecting sums for certain ids, we sum up everything we have.
        mys[unique_ids[i]] = sum(amounts)
        ### If we have denominators, we also sum them up.
        if weighting:
            if len(denominators) > 0:
                myd[unique_ids[i]] = sum(denominators)
#
        i+=1
    ### Let's update the records from previous reputations and how we update them (for raters)
    for k in prev_rep1a.keys():
        previous_rep[k] = prev_rep1a[k]
    ### If we have weighting and denomination, then we 
    if weighting:
        if denomination and len(mys) == len(myd):
            for k, v in mys.items():
                ### divide mys values with denomination values.
                mys[k] = v / myd[k]
    ### nr 5.
    ### Here we make trasformation in the same way as described in point 5 in documentation doc.
    text = "Differential before log transformation: " + str(mys)
    logging.debug(text)
    if logranks:
        for k in mys.keys():
            if mys[k]<0:
                mys[k] = -np.log10(1 - mys[k])
            else:
                mys[k] = np.log10(1 + mys[k])
    logging.debug(text)
    return(mys,previous_rep)

### normalizing differential.
def normalized_differential(mys,normalizedRanks,our_default,spendings,log=True):
    ### Nr 6;
    ### We divide it by max value, as specified. There are different normalizations possible...
    ### lograting transformation as made by Anton. Since this was done few lines above, I believe this is redundant coding.
    if log:
        for k in mys.keys():
            mys[k] = -np.log10(1 - mys[k]) if mys[k] < 0 else np.log10(1 + mys[k])
    ### It could as well be deleted; in this case test_spendings_normalization wll have different result.        
            
    ### Then we use maximums, either from what we have or set it to one by default.
    max_value = max(mys.values(), default=1)
    min_value = min(mys.values(), default=0)
    if max_value==0: #normalized zeroes are just zeroes
    	return(mys)
    ### Now some rare cases, such as we have only one value of differential and what to do then.
    if max_value==min_value:
        min_value = max_value - our_default ### as the solution to issue #157
        if min_value==max_value and spendings>0:
            min_value = max_value - 1
        
    ### Now, way of solving this problem in a bit more common way:        
    for k in mys.keys():
        if max_value==min_value:
            ### Still looking at a special case when max_value==min_value.
            mys[k] = (mys[k]-min_value)
        else:
            if normalizedRanks: ### normalizedRanks is equal to fullnorm.
                ### Then we normalized based on whether we have normalizedRanks or not.
                mys[k] = (mys[k]-min_value) /(max_value-min_value)
            else:
                mys[k] = mys[k] /max_value 

    return(mys)   
 
### Get updated reputations, new calculations of them...
### This one is with log...

def rater_reputation(previous_reputations,rater_id,default,prev_reputation,liquid=False,to_id = [],predictiveness = 0,predictive_data = dict()):
    ### Assigning rater reputation. It is not trivial; if liquid=True, then we can expect that 
    if rater_id in previous_reputations.keys():
        
        ### Checks whether it's liquid or not. If liquid, return 1, otherwise previous reputation.
        if (not liquid):
            rater_rep = 1
        else:
            if rater_id in prev_reputation:
                rater_rep = previous_reputations[rater_id] * 100
            else:
                rater_rep = previous_reputations[rater_id] * 100
    else:

        if (not liquid):
            rater_rep = 1
        else:
            rater_rep = default * 100
        ### If it is not in reputations up to the current one, we set a default value.

    if predictiveness>0:
        if rater_id not in predictive_data.keys():
            pass # Do nothing
            #raise ValueError('Calling for predictiveness without previous predictive data.')
        else:
            rater_rep = rater_rep * (1-predictiveness) + predictiveness * predictive_data[rater_id] * rater_rep
    previous_rep1 = dict()
    for k in prev_reputation.keys():
        previous_rep1[k] = prev_reputation[k]
    previous_rep1[rater_id] = rater_rep
    return(rater_rep,previous_rep1)

### Another normalization. This one is intended for reputation normalization.
def normalize_reputation(reputation,new_array,unrated,default1,decay,conservatism,normalizedRanks=False):
    max_value = max(reputation.values(), default=1)
    min_value = min(reputation.values(), default=0)
    ### First we make the same max/min values.
    for k in reputation.keys():
        if normalizedRanks: ### normalizedRanks is equal to fullnorm.
            if max_value!=min_value:
                reputation[k] = (reputation[k]-min_value) /(max_value-min_value)
            else:
                pass
        else:
        ### Now, way of solving this problem in a bit more common way:        
            if max_value!= 0:
                reputation[k] = reputation[k] /max_value
            else:
                pass
    i = 0
    ### if unrated=False, we discount new agents for conservativity and decay.
    while i<len(new_array):
        if unrated:
            if new_array[i][0] in reputation.keys():
                pass
            else:
                reputation[new_array[i][0]] = conservatism * default1 + (1-conservatism) * decay
        i+=1
    return(reputation)    
    

### Initialize dictionary with all keys from our dataset and 0 values;
def initialize_dict(from_array,to_array):
    mydict = {}
    for k in np.unique(from_array):
        if k in mydict.keys():
            pass
        ## If we do not have this record, we set it default reputation.
        else:
            mydict[str(k)] = 0
    for k in np.unique(to_array):
        if k in mydict.keys():
            pass
        ## If we do not have this record, we set it default reputation.
        else:
            mydict[str(k)] = 0
    return(mydict)

### Updating reputation - blending differential and reputation.
### In original paper, there were a few proposed ways of updating and approach d has been found to be the most
### useful and the only ne we are using at the moment.
def update_reputation_approach_d(first_occurance,reputation,mys,since,our_date,default_rep,conservativity):
    ### Our current approach of updating reputation each time period. 
    j = 0
    all_keys = set(mys.keys())
    for k  in reputation.keys():
        if k in all_keys:
            ### for everything inside reputation and differential, this is the equation we a reusing-
            reputation[k] = (1-conservativity) * mys[k] + conservativity * reputation[k]
        else:
            ### when in reputation, but not in differential, this is what we do:
            reputation[k] = (1-conservativity) * default_rep + conservativity * reputation[k]
        j+=1  
    return(reputation)



### spendings_based function. So, we look at 'from' transactions and use the same calculation as 
### for normal differential, except that we use it for 'from' ids.
def spending_based(transactions,som_dict,logratings,precision,weighting):
    i=0
    while i<len(transactions):
        if transactions[i][0] in som_dict.keys():
            if not weight_calc(transactions[i],logratings,precision,weighting)[1]==None:
                som_dict[transactions[i][0]] += weight_calc(transactions[i],logratings,precision,weighting)[1]
            else:
                som_dict[transactions[i][0]] += weight_calc(transactions[i],logratings,precision,weighting)[0]
                ### Not sure about above fix, but sometimes we have none value if weighting=False. This should fix it...
        else:
            if not weight_calc(transactions[i],logratings,precision,weighting)[1]==None:
                som_dict[transactions[i][0]] = weight_calc(transactions[i],logratings,precision,weighting)[1]### changed from
            #### new_rating instead of new_weight.
            else:
                som_dict[transactions[i][0]] = weight_calc(transactions[i],logratings,precision,weighting)[0]
        i+=1
    return(som_dict)

### An alternative to np.where - created because sometimes there could be problems with former.
def where(to_array,the_id):
    our_ids = []
    i=0
    while i<len(to_array):
        if to_array[i]==the_id:
            our_ids.append(i)
        i+=1
    return(our_ids)

### average_individual_rating_by_period
def calculate_average_individual_rating_by_period(transactions,weighted):
    count_trans = dict()
    #if weighted:
    ratings_avg = dict()
    i = 0
    while i<len(transactions):
        if transactions[i][0] in ratings_avg.keys():
            if transactions[i][1] in ratings_avg[transactions[i][0]].keys():
                ratings_avg[transactions[i][0]][transactions[i][1]].append(transactions[i][3]) ### should be value append.
                count_trans[transactions[i][0]][transactions[i][1]] += 1
            else:
                ratings_avg[transactions[i][0]][transactions[i][1]] = [transactions[i][3]]
                count_trans[transactions[i][0]][transactions[i][1]] = 1
        else:
            ratings_avg[transactions[i][0]] = dict()
            count_trans[transactions[i][0]] = dict()
            ratings_avg[transactions[i][0]][transactions[i][1]] = [transactions[i][3]]
            count_trans[transactions[i][0]][transactions[i][1]] = 1
        i+=1
    ### Now we make averages over everything.
    for k in ratings_avg.keys():
        for j in ratings_avg[k].keys():
            ratings_avg[k][j] = np.mean(ratings_avg[k][j])
    return(ratings_avg,count_trans)

def max_date(mydict):
    ### Get dictionary where keys are dates and we get the value of last date;
    sorted_days = sorted(mydict.keys())
    last_date = sorted_days[-1]
    i = 0
    return(mydict[last_date])

### function of predictiveness
def update_predictiveness_data(previous_pred,mydate,reputations,transactions,conservatism):
    ids_used = []
    for k in transactions:
        from_id = k
        ids_used.append(k)
        if from_id not in previous_pred.keys():
            previous_pred[from_id] = dict()
        for to_id in transactions[from_id]:
            if to_id in previous_pred[from_id].keys():
                previous_pred[from_id][to_id] = transactions[from_id][to_id] * (1-conservatism) + conservatism * previous_pred[from_id][to_id] ### mydate should not exist yet in our run.
            else:
                previous_pred[from_id][to_id] = dict()
                previous_pred[from_id][to_id] = transactions[from_id][to_id] #
    return(previous_pred,ids_used)

def normalize_individual_data(mydate,new_ids):
    all_from = new_ids.keys()
    max_values = dict()
    for k in all_from.keys():
        max_values[k] = []
        for j in all_from[k].keys():
            if mydate in all_from[k][j].keys():
                max_values.append(all_from[k][j][mydate])
        
        ### ok, now we've added values to max_values. We continue with another, similar loop;
        max_value = max(max_values)
        for j in all_from[k].keys():
            if mydate in all_from[k][j].keys():
                all_from[k][j][mydate] = all_from[k][j][mydate]/max_values

                
def calculate_distance(previous_individual_rating,curret_reputation_rank):
    distance = 0
    j = 0
    while j<len(previous_individual_rating):
        distance += (curret_reputation_rank[j]/1 - previous_individual_rating[j]/1)**2
        j+=1
    distance = distance/len(previous_individual_rating)
    return(np.sqrt(distance))

def normalize_correlations(mydict):
    mymax = max(mydict.values())
    for k in mydict.keys():
        mydict[k] = mydict[k]/mymax
    return(mydict)

In [4]:
# Reputation Scenario:



# Simulation Reputation Scenario Data Generator

import random
import datetime
import time

network = 'reptest'

# percents of goodness for agent to be selected by consumer, 0 to select all, 100 or None to select only the top
#threshold = 10 # for "10 agents X 10 days", that is too low - bad agents still get chance to be selected on days 2 and 3
#threshold = 50 # for "10 agents X 10 days", that is too high - one good agent gets associated with bad agents
threshold = 40

def list_best_ranked(ranks,list,threshold=None):
	if threshold is None:
		threshold = 0
		for key in ranks:
			value = ranks[key]
			if threshold < value:
				 threshold = value
	if threshold == 0:
		return list
	best = []
	for item in list:
		key = str(item)
		if key in ranks:
			value = ranks[key]
			if threshold <= value:
				 best.append(item)
	if len(best) == 0:
		return list
	return best

def intersection(lst1, lst2): 
	lst3 = [value for value in lst1 if value in lst2] 
	return lst3

def pick_agent(ranks,list,self,memories = None,bad_agents = None):
	picked = None
	if memories is not None:
		#good agents case
		if self in memories:
			blacklist = memories[self]
		else:
			blacklist = []
			memories[self] = blacklist
		if blacklist is not None:
			list = [white for white in list if white not in blacklist and white != self]
		if ranks is not None:
			list = list_best_ranked(ranks,list,threshold)
	else:
		#bad agents case
		blacklist = None
		list = [black for black in list if black != self]
	picked = list[0] if len(list) == 1 else list[random.randint(0,len(list)-1)]

	#debug
	#if blacklist is not None:
	#	print(picked)
	
	if blacklist is not None and bad_agents is not None and picked in bad_agents:
		blacklist.append(picked) #blacklist picked bad ones once picked so do not pick them anymore
	return picked


def log_file(file,date,type,agent_from,agent_to,cost,rating):
	timestr = str(time.mktime(date.timetuple()))
	timestr = timestr[:-2] # trim .0 part
	file.write(network + '\t' + timestr + '\t' + type + '\t' + str(agent_from) + '\t' + str(agent_to) + '\t' + str(cost) + '\t' \
			+ '\t\t\t\t\t\t\t\t' + ('' if rating is None else str(rating)) + '\t\n')


def get_list_fraction(list,fraction,first):
	n = round (len(list) * (fraction if first else 1 - fraction))
	res_list = list[:n] if first else list[n:]
	return res_list


def reputation_simulate(good_agent,bad_agent,since,sim_days,ratings,rs,verbose=False,campaign=None,silent=False):
	random.seed(1) # Make it deterministic
	memories = {} # init blacklists of compromised ones

	if rs is not None:
		rs.clear_ratings()
		rs.clear_ranks()

	actual_bad_volume = 0
	actual_good_volume = 0
	actual_good_to_bad_volume = 0
	
	good_agents = [i for i in range(good_agent['range'][0],good_agent['range'][1]+1)]
	bad_agents = [i for i in range(bad_agent['range'][0],bad_agent['range'][1]+1)]

	good_suppliers = get_list_fraction(good_agents,good_agent['suppliers'],True)
	bad_suppliers = get_list_fraction(bad_agents,bad_agent['suppliers'],True)
	good_consumers = get_list_fraction(good_agents,good_agent['consumers'],False)
	bad_consumers = get_list_fraction(bad_agents,bad_agent['consumers'],False)
	all_suppliers = good_suppliers + bad_suppliers
	all_consumers = good_consumers + bad_consumers
	all_agents = good_agents + bad_agents

	if verbose:
		print('Good:',good_agent)
		print('Bad:',bad_agent)
		print('Good:',good_agents)
		print('Bad:',bad_agents)
		print('All suppliers:',all_suppliers)
		print('All consumers:',all_consumers)
		print('Good suppliers:',good_suppliers)
		print('Good consumers:',good_consumers)
		print('Bad suppliers:',bad_suppliers)
		print('Bad consumers:',bad_consumers)
	
	good_agents_transactions = good_agent['transactions']
	bad_agents_transactions = bad_agent['transactions']
	good_agents_values = good_agent['values']
	bad_agents_values = bad_agent['values']
	good_agents_count = len(good_agents)
	bad_agents_count = len(bad_agents)
	good_agents_volume = good_agents_count * good_agents_transactions * good_agents_values[0]
	bad_agents_volume = bad_agents_count * bad_agents_transactions * bad_agents_values[0]
	code = ('r' if ratings else 'p') + '_' + str(round(good_agents_values[0]/bad_agents_values[0])) + '_' + str(good_agents_transactions/bad_agents_transactions) \
		+ (('rs' if rs.get_parameters()['weighting'] == True else 'nw') if rs is not None else '') 
	transactions = 'transactions' + str(len(all_agents)) + '_' + code + '.tsv'
	
	if verbose:
		print('Good:',len(good_agents),good_agents_values[0],good_agents_transactions,len(good_agents)*good_agents_values[0]*good_agents_transactions)
		print('Bad:',len(bad_agents),bad_agents_values[0],bad_agents_transactions,len(bad_agents)*bad_agents_values[0]*bad_agents_transactions)
		print('Code:',code,'Volume ratio:',str(good_agents_volume/bad_agents_volume))

	with open(transactions, 'w') as file:
		for day in range(sim_days):
			prev_date = since + datetime.timedelta(days=(day-1))
			date = since + datetime.timedelta(days=day)
			if verbose:
				print(day,date,memories)

			if rs is not None:
				#update ranks for the previous day to have them handy
				rs.update_ranks(prev_date)
				ranks = rs.get_ranks_dict({'date':prev_date})
				if verbose:
					print('Ranks',ranks)
			else:
				ranks = None

			#resetting scam campaign
			skip_scam = False
			if campaign is not None:
				campaign_day = day % campaign[0] # campaign.period
				if campaign_day == 0:
					if day > 0:
						if verbose:
							print('reset scam')
						scam_generation = day // campaign[0] # campaign.period
						id_base = len(bad_agents) * scam_generation
						bad_agents = [i+id_base for i in range(bad_agent['range'][0],bad_agent['range'][1]+1)]
						bad_suppliers = get_list_fraction(bad_agents,bad_agent['suppliers'],True)
						bad_consumers = get_list_fraction(bad_agents,bad_agent['consumers'],False)
						all_suppliers = good_suppliers + bad_suppliers
					if verbose:
						print('bad agents:',bad_agents)
				if campaign_day < campaign[1]: # campaign.inactive
					if verbose:
						print('skip scam')
					skip_scam = True
				if verbose:
					print('do scam')

			daily_good_to_bad = 0
			for agent in good_consumers:
				daily_selections = {}
				for t in range(0, good_agents_transactions):
					other = pick_agent(ranks,all_suppliers,agent,memories,bad_agents)
					cost = random.randint(good_agents_values[0],good_agents_values[1])
					actual_good_volume += cost
					# while ratings range is [0.0, 0.25, 0.5, 0.75, 1.0], we rank good agents as [0.25, 0.5, 0.75, 1.0]
					rating = 0.0 if other in bad_agents else float(random.randint(1,4))/4
					if other in bad_agents:
						actual_good_to_bad_volume += cost
						daily_good_to_bad += cost
					if ratings:
						if rs is not None:
							rs.put_ratings([{'from':agent,'type':'rating','to':other,'value':rating,'weight':cost,'time':date}])
						log_file(file,date,'rating',agent,other,rating,cost)
					else: 
						if rs is not None:
							rs.put_ratings([{'from':agent,'type':'transfer','to':other,'value':cost,'time':date}])
						log_file(file,date,'transfer',agent,other,cost,None)
					daily_selections[other] = 1 + daily_selections[other] if other in daily_selections else 1
				if verbose:
					print('By ' + str(agent) + ':' + str(daily_selections))

			if verbose:
				print(daily_good_to_bad)

			if skip_scam:
				continue
		
			for agent in bad_consumers:
				for t in range(0, bad_agents_transactions):
					other = pick_agent(None,bad_suppliers,agent)
					cost = random.randint(bad_agents_values[0],bad_agents_values[1])
					actual_bad_volume += cost
					if ratings:
						rating = 1.0
						if rs is not None:
							rs.put_ratings([{'from':agent,'type':'rating','to':other,'value':rating,'weight':cost,'time':date}])
						log_file(file,date,'rating',agent,other,rating,cost)
					else: 
						if rs is not None:
							rs.put_ratings([{'from':agent,'type':'transfer','to':other,'value':cost,'time':date}])
						log_file(file,date,'transfer',agent,other,cost,None)
					
	with open('users' + str(len(all_agents)) + '.tsv', 'w') as file:
		for agent in all_agents:
			goodness = '0' if agent in bad_agents else '1'
			file.write(str(agent) + '\t' + goodness + '\n')

	if verbose:
		print('Actual volumes and ratios:')

	def ratio_str(x,y):
		return 'INF' if y == 0 else x/y
		
	if silent is not True:
		print('Good:',str(actual_good_volume),'Bad:',str(actual_bad_volume),'Good to Bad:',actual_good_to_bad_volume,'Good/Bad:',ratio_str(actual_good_volume,actual_bad_volume),'Bad/Good_to_Bad:',ratio_str(actual_bad_volume,actual_good_to_bad_volume),'LTS:',ratio_str(actual_good_to_bad_volume,actual_good_volume),'PFS:',ratio_str(actual_good_to_bad_volume,actual_bad_volume))


In [5]:
# Reputation Service API, including Rating Service and Ranking Service




from datetime import datetime, timedelta,date
#from reputation_base_api import ReputationServiceBase
import logging as logging

"""
Reputation Service native implementation in Python
"""  


class PythonReputationService(ReputationServiceBase):
    
    ### This function allows us to set up the parameters.
    ### Setting up the way we do in Anton's recommendation.
    ### update_period is how many days we jump in one period... We can adjust it...

    ### Set parameters. In changes, there is a dictionary with values and if there are no determined values,
    ### we set up default values.
    def set_parameters(self,changes):
        if 'default' in changes.keys():
            self.default = changes['default']
        else:
            if 'default' in dir(self):
                pass
            else:
                self.default = 0.5 # default value 
        if 'conservatism' in changes.keys():
            self.conservatism = changes['conservatism']
        else:
            if 'conservatism' in dir(self):
                pass
            else:
                self.conservatism = 0.5
        if 'precision' in changes.keys():
            self.precision = changes['precision']
        else:
            if 'precision' in dir(self):
                pass
            else:
                self.precision = 0.01
        if 'weighting' in changes.keys():
            self.weighting = changes['weighting']
        else:
            if 'weighting' in dir(self):
                pass
            else:
                self.weighting = True
        if 'denomination' in changes.keys():
            self.denomination = changes['denomination']
        else:
            if 'denomination' in dir(self):
                pass
            else:
                self.denomination = False
        if 'fullnorm' in changes.keys():
            self.fullnorm = changes['fullnorm']
        else:
            if 'fullnorm' in dir(self):
                pass
            else:
                self.fullnorm = True
        if 'liquid' in changes.keys():
            self.liquid = changes['liquid']
        else:
            if 'liquid' in dir(self):
                pass
            else:
                self.liquid = True
        if 'logranks' in changes.keys():
            self.logranks = changes['logranks']
        else:
            if 'logranks' in dir(self):
                pass
            else:
                self.logranks = True
        if 'update_period' in changes.keys():
            self.update_period = changes['update_period']
        else:
            if 'update_period' in dir(self):
                pass
            else:
                self.update_period = 1
        if 'logratings' in changes.keys():
            self.logratings = changes['logratings']
        else:
            if 'logratings' in dir(self):
                pass
            else:
                self.logratings = False
        if 'temporal_aggregation' in changes.keys():
            self.temporal_aggregation = changes['temporal_aggregation']
        else:
            if 'temporal_aggregation' in dir(self):
                pass
            else:
                self.temporal_aggregation = False
        if 'use_ratings' in changes.keys():
            self.use_ratings = changes['use_ratings']
        else:
            if 'use_ratings' in dir(self):
                pass
            else:
                self.use_ratings = True
        if 'start_date' in changes.keys():
            start_date = changes['start_date']
            self.date = start_date
        else:
            if 'date' in dir(self):
                pass
            else:
                self.date = date(2018, 1, 1)       
        if 'first_occurance' in changes.keys():
            self.first_occurance = changes['first_occurance']
        else:
            if 'first_occurance' in dir(self):
                pass
            else:
                self.first_occurance = {}
        if 'decayed' in changes.keys():
            self.decayed = changes['decayed']
        else:
            if 'decayed' in dir(self):
                pass
            else:
                self.decayed = 0
        if 'downrating' in changes.keys():
            self.downrating = changes['downrating']
        else:
            self.downrating = False   
        if 'unrated' in changes.keys():
            self.unrated = changes['unrated']
        else:
            self.unrated = False   
        if 'spendings' in changes.keys():
            self.spendings = changes['spendings']
        else:
            self.spendings = 0.0   
        if 'ratings' in changes.keys():
            self.ratings_param = changes['ratings']
        else:
            self.ratings_param = 1.0      
        if 'rating_bias' in changes.keys():
            self.rating_bias = changes['rating_bias']
        else:
            self.rating_bias = False
        if 'predictiveness' in changes.keys():
            self.predictiveness = changes['predictiveness']
        else:
            self.predictiveness = 0   
        if 'logging' in changes.keys():
            self.logging = changes['logging']
        else:
            self.logging = True              
        if self.logging: ### If logging is enabled, we log everything
            log_name = "python-log" + datetime.datetime.now().strftime('%Y-%m-%d') + "-log.log"
            logging.basicConfig(filename=log_name, filemode='w', format='%(name)s - %(levelname)s - %(asctime)s - %(message)s', level=10)
            logging.info('We start our system.')
        else:### If logging is disabled, we log only critical messages or higher priority messages.
            log_name = "python-log_critical" + datetime.datetime.now().strftime('%Y-%m-%d') + "-log.log"
            logging.basicConfig(filename=log_name, filemode='w', format='%(name)s - %(levelname)s - %(asctime)s - %(message)s',level=logging.CRITICAL)
            logging.disable(logging.CRITICAL)
            logging.info('We start our system.') 
        text = "set parameters default " + str(self.default) + ", decayed " + str(self.decayed) + ", conservatism "+str(self.conservatism) + ", precision " + str(self.precision) + ", liquid "+str(self.liquid) + ", update_period " + str(self.update_period) + ", aggregation " + str(self.temporal_aggregation) + ", downrating " + str(self.downrating) + ", fullnorm " + str(self.fullnorm) + ", weighting " + str(self.weighting) + ", denomination " + str(self.denomination) + ", logratings " + str(self.logratings) + ", ratings " + str(self.ratings_param) + ", spendings " + str(self.spendings) + ", predictiveness " + str(self.predictiveness) + ", unrated " + str(self.unrated) + ", rating_bias " + str(self.rating_bias) + ", logranks " + str(self.logranks)
        logging.debug(text)    
        return(0)
        
    ### This functions merely displays the parameters.
    def get_parameters(self):              
        return({'default': self.default,'decayed':self.decayed, 'conservatism':self.conservatism, 'precision':self.precision,
                'liquid':self.liquid,'update_period':self.update_period,'aggregation':self.temporal_aggregation,
                'downrating':self.downrating,'fullnorm':self.fullnorm, 'weighting':self.weighting,'denomination':self.denomination,
                'logratings':self.logratings, 'ratings':self.ratings_param, 'spendings':self.spendings,'predictiveness':self.predictiveness,'unrated':self.unrated,'rating_bias':self.rating_bias,'logranks':self.logranks})
#27:body:2019-07-30T23:53:21.511:I:8c5b4f5fb913462e99c248daccfc36c0:reputation network test set parameters default 0.5 decayed 0.5 conservatism 0.25 precision 0.01 liquid true period 1 aggregation true downrating false fullnorm false weighting true denomination false logratings false ratings 1.0 spendings 0.0 parents 0.0 predictiveness 1 rating_bias true unrated false
    
    
    
    ## Update date
    def set_date(self,newdate):
        self.our_date = newdate
    ### Clear stored reputations.    
    def clear_ranks(self):
        self.reputation = {}   
        self.all_reputations = {}
        self.predictive_data = dict()
        self.pred_values = dict()
        self.count_values = dict()
        logging.debug('Ranks cleared.')
        return(0)
    ### Clear stored ratings (transactions).
    def clear_ratings(self):
        self.ratings = {}
        logging.debug('Ratings cleared.')
        return(0)
    ### Initialization of reputation.    
    def initialize_ranks(self,reputation=None,first_occurance = None):
        ### First, we check if there is a reputation dictionary (where ranks should be stored) and if not, create one.
        if reputation==None:
            self.reputation = dict()
        else:
            self.reputation = reputation      
    ### get all dates that we have in stored ratings. This is how we can store them in self.dates.
    def all_dates(self):
        i = 0
        all_dates = []
        while i<len(self.ratings):
            if self.ratings[i]['time'] in all_dates:
                pass
            else:
                all_dates.append(self.ratings[i]['time'])
            i+=1
        self.dates = all_dates
    ### select ratings from selected update period (whichever it is set).
    def select_data(self,mydate):
        min_date = mydate - timedelta(days=self.update_period) # We look today minus update_period number of days.
        max_date = mydate
        i=0
        while i<len(self.ratings):
            mydict = self.ratings[i]
            if type(mydict) is list:
                mydict = mydict[0]
            if (mydict['time'] > min_date and mydict['time']<=max_date): ## If in right ime period, then we add it to
                ### current_ratings. Which we analyze.
                self.current_ratings.append(mydict)
            i+=1
   
    ### We can also convert data from pandas to dictionary.
    def convert_data(self,data):
        daily_data = data[['from','type','to','weight','time','value','type']].to_dict("records")
        self.all_dates()
        i = 0
        while i<len(daily_data):
            daily_data[i]['from'] = str(daily_data[i]['from'])# the only thing we make sure is that
            # 'from' and 'to' are strings.
            daily_data[i]['to'] = str(daily_data[i]['to'])
            i+=1
        self.ratings = daily_data
        return(daily_data)

    ### We run the update in this function.    
    def update_ranks(self,mydate):
        if not "rater_ranks_special" in dir(self):
            self.rater_ranks_special = dict()
        self.non_rounded_rep = dict()
        text = "period " +  str(self.update_period)
        logging.debug(text)
        ### And then we iterate through functions. First we prepare arrays and basic computations.
        since = mydate - timedelta(days=self.update_period)
        if self.rating_bias:
            if 'rater_biases' in dir(self):
                pass
            else:
                self.rater_biases = dict()
                self.rater_biases[since] = dict()
        if self.predictiveness>0:
            if 'predictive_data' in dir(self):
                pass
            else:
                self.predictive_data = dict()
                self.pred_values = dict()
                self.count_values = dict()
                
        self.current_ratings = []
        ### Sellect data which we will use.
        self.select_data(mydate)
        i=0
        problem = False
        while i<len(self.current_ratings):
            ### If we do not have values, then we have a problem. Even if only one rating is missing,
            ### there is a problem.
            if (self.use_ratings==True and (not 'value' in self.current_ratings[i].keys())):
                problem=True
            i+=1
        if problem:
            ### Well, if we have a problem, we just set ratings to false. Code will still work.
            logging.warning("Ratings is set to True, but no ratings were given. Changing the setting to False")
            self.use_ratings=False
        problem = False 
        ### if we have some ratings, we can check;
        if len(self.current_ratings)>0:
            ### If we have payments and downratings to true, there is a small error.
            if (self.current_ratings[0]['type']=="payment" and self.downrating==True):
                ### raising error message.
                warnings.warn("if we only have payments, we have no ratings. Therefore downratings cannot be True. Setting them to False") 
                self.downrating=False
        ### we set up arrays; this is the set of data where we have ratings, values, weights
        ### in predictable way, so we can iterate them later on.
        if self.rating_bias:
            if 'average_ratings' in dir(self):
                pass
            else:
                self.average_ratings = dict()
            self.rater_biases[mydate] = dict()
            array1 , dates_array, to_array, rater_biases1, avgs = reputation_calc_p1(self.current_ratings,self.conservatism,self.precision,self.temporal_aggregation,False,self.logratings,self.downrating,self.weighting,self.rater_biases[since],self.average_ratings)
            self.average_ratings[mydate] = avgs
            self.rater_biases[mydate] = dict(rater_biases1)
        else:
            array1 , dates_array, to_array, rater_biases1 = reputation_calc_p1(self.current_ratings,self.conservatism,self.precision,self.temporal_aggregation,False,self.logratings,self.downrating,self.weighting, None)  ### In this case, we don't need rater_biases
        ### Now we update the reputation. Here, old ranings are inseter and then new ones are calculated as output.

        self.reputation = update_reputation(self.reputation,array1,self.default,self.spendings)
        
        
        ### If we have spendings-based reputation, we go in the loop below.
        if self.spendings>0:
            spendings_dict = spending_based(array1,dict(),self.logratings,self.precision,self.weighting)
            ### We normalize differential that is spendings-based.
            spendings_dict = normalized_differential(spendings_dict,normalizedRanks=self.fullnorm,our_default=self.default,spendings=self.spendings,log=self.logranks)     
        
        ### Then we calculate differential the normal way.
        if self.predictiveness>0:
            new_reputation,self.rater_ranks_special = calculate_new_reputation(logging = logging,new_array = array1,to_array = to_array,reputation = self.reputation,rating = self.use_ratings,precision = self.precision,previous_rep = self.rater_ranks_special,default=self.default,unrated=self.unrated,normalizedRanks=self.fullnorm,weighting = self.weighting,denomination = self.denomination, liquid = self.liquid, logratings = self.logratings,logranks = self.logranks, predictiveness = self.predictiveness,predictive_data = self.pred_values)
        else:
            new_reputation,self.rater_ranks_special = calculate_new_reputation(logging = logging,new_array = array1,to_array = to_array,reputation = self.reputation,rating = self.use_ratings,precision = self.precision,previous_rep = self.rater_ranks_special,default=self.default,unrated=self.unrated,normalizedRanks=self.fullnorm,weighting = self.weighting,denomination = self.denomination, liquid = self.liquid, logratings = self.logratings,logranks = self.logranks, predictiveness = self.predictiveness)
        ### And then we normalize the differential:
        new_reputation = normalized_differential(new_reputation,normalizedRanks=self.fullnorm,our_default=self.default,spendings=self.spendings,log=False)
        text = "Normalized differential: " + str(new_reputation)
        logging.debug(text)  
        ### Again only starting this loop if we have spendings.
        ### we take data from date-update_period.
        ### For spendings-based system. We store all agents' reputations and then we only show reputations of
        ### sellers when get_ranks() is called. This is not in line with Java implementation, which only returns
        ### sellers. To circumvent that, we make an object in which we store only sellers and we then return only sellers.
        ### We still need all ranks, so they are still stored in self.all_reputations
        
        if "sellers" in dir(self):
            pass
        else:
            self.sellers = []
        
        for k in new_reputation.keys():
            if k not in self.sellers:
                self.sellers.append(k)
        if (self.spendings>0 and self.predictiveness==0):
            updated_differential = dict()
            unique_keys = list(new_reputation.keys())
            ###  each 'from' is added to unique_keys list. We add it to what is already in differential.
            for k in spendings_dict.keys():
                if not k in unique_keys:
                    unique_keys.append(k)
            for k in unique_keys:
                ### Then we have different cases of what happens depending on where we have a certain key.
                if (k in new_reputation.keys()) and (k in spendings_dict.keys()):
                    ### Note, everything is already nomalized. The rest are just the equations to make sure everything is correct.
                    updated_differential[k] = (self.ratings_param * new_reputation[k] + self.spendings * spendings_dict[k])/ (self.spendings + self.ratings_param)
                if (k in new_reputation.keys()) and (k not in spendings_dict.keys()): 
                    updated_differential[k] = (self.ratings_param * new_reputation[k])/ (self.spendings + self.ratings_param)
                if (k not in new_reputation.keys()) and (k in spendings_dict.keys()): 
                    updated_differential[k] = (self.spendings * spendings_dict[k])/ (self.spendings + self.ratings_param)
            ### Differential is then from both spendings and usual differential.
            new_reputation = updated_differential
        # THen we blend the reputation with differential.
        self.reputation = update_reputation_approach_d(self.first_occurance,self.reputation,new_reputation,since,
                                                       self.date, self.decayed,self.conservatism)
        ### Apply normalizedRanks=True AKA "full normalization" to prevent negative ratings on "downrating"
        ### See line 360 in https://github.com/aigents/aigents-java/blob/master/src/main/java/net/webstructor/peer/Reputationer.java
        ### and line 94 in https://github.com/aigents/aigents-java/blob/master/src/main/java/net/webstructor/data/Summator.java 
        ### Downratings seem to pass, so I assume this comment is resolved.
        text = "Blended differential with reputation: " + str(self.reputation)
        logging.debug(text) 
        self.reputation = normalize_reputation(self.reputation,array1,self.unrated,self.default,self.decayed,self.conservatism,self.downrating)
        self.non_rounded_rep = dict(self.reputation)
        ### round reputations:
        for k in self.reputation.keys():
            self.reputation[k] = my_round(self.reputation[k],2) # Make sure we use my_round. 
            ### This might be changed in the future, but now rounding is done in order to be the same as in Java rs.
        ## We have all_reputations dictionary where we have all history of reputations with dates as keys.
        text = "Normalized reputation: " + str(self.reputation)
        logging.debug(text)
        self.all_reputations[mydate] = dict(self.reputation)
        if self.predictiveness>0:
            avg_ind_rat_byperiod,self.count_values[mydate] = calculate_average_individual_rating_by_period(array1,True)
            text = "Average individual rating by period: " + str(avg_ind_rat_byperiod)
            logging.debug(text)
            #self.predictive_data, ids = update_predictiveness_data(self.predictive_data,mydate,self.reputation,avg_ind_rat_byperiod,self.conservatism)
            self.predictive_data, ids = update_predictiveness_data(self.predictive_data,mydate,self.reputation,avg_ind_rat_byperiod,self.conservatism)
            self.calculate_indrating(ids,mydate)
            text = "Individual rating: " + str(self.predictive_data) + ", and rating used for rater reputation: " + str(self.pred_values)
            logging.debug(text)                 
        return(0)

    ### When we want to save ratings to our system. So, we can add them, the same way as in Java.        
    ### Except that we can add many of them at once.
    def put_ratings(self,ratings):
        i = 0
        while i<len(ratings):
            # For each of the ratings that we want to add, we transform from and to to string, just in case.
            # 
            ratings[i]['from'] = str(ratings[i]['from'])
            ratings[i]['to'] = str(ratings[i]['to'])
             
            i+=1
        ### if we have no ratings yet in our system, we add all ratings from ground up.
        if self.ratings == {}:
            self.ratings = ratings
        else:
            self.ratings.append(ratings)
        text = "Adding ratings: " + str(ratings)
        logging.debug(text)  
        return(0)  
    ### This is how we get current ranks. times are basically expecting dates
    ### when those ranks were calculated.
    def get_ranks(self,times):
        ### If we don't have reputations yet, we return empty array. We start with empty dictionary if there are no reputations.
        if self.all_reputations == {}:
            result = {}
        else:
            ### If there were previous reputations, we look if date we are looking for has reputations. If so, we save them
            ### in result object.
            if times['date'] in list(self.all_reputations.keys()):
                result = dict(self.all_reputations[times['date']])
            else:
                ### If there are no reputations for that date, we work with empty dictionary.
                result = {}
        all_results = []
        ### In the end we only round up the ranks when we return them.
        for k in result.keys():
            all_results.append({'id':k,'rank':my_round(result[k]*100,0)})  
        #logging.debug("network get ranks: ",str(all_results))
        logging.info("network get ranks: {0}".format(all_results))
        ### Now, if we have spending, we only return those that are sellers;
        #if self.spendings>0:
        #    my_results = dict()
        #    for k in all_results.keys():
        #        if k in self.sellers:
        #            my_results[k] = all_results[k]       
        #    all_results=my_results                    
        return(0,all_results)
    ### get_ranks_dict is similar as get_ranks
    def get_ranks_dict(self,times):
        if self.all_reputations == {}:
            result = {}
        else:
            if times['date'] in list(self.all_reputations.keys()):
                result = dict(self.all_reputations[times['date']])
            else:
                result = {}
        for k in result.keys():
            result[k] = my_round(result[k]*100,0)    
            ### Everything is similar to get_ranks, but we only return result, not really 0 beside result.
        #logging.debug("network get ranks: " , str(result))
        logging.info("network get ranks: {0}".format(result))
        #if self.spendings>0:
        #    my_results = dict()
        #    for k in result.keys():
        #        if k in self.sellers:
        #            my_results[k] = result[k]
        #    result=my_results
        return(result)    

    ### Getting ratings from Python rs.
    def get_ratings(self,times={}):
        ### we need dates as input. In case no dates are given, we return 0.
        if times=={}:
            return(0,self.ratings)
        else:
            ### If we have dates, then we first make sure that ids are strings.
            all_ids = str(times['ids'])
            since = times['since']
            until = times['until']
            results = []
            i = 0
            while i<len(self.ratings):
                ### We loop over all ratings. In some cases, ratings are lists, in this case we "unlist" them.
                if type(self.ratings[i]) is list:
                    self.ratings[i] = self.ratings[i][0]
                ### We also look at from ids and then just make sure they are in the right time frame. If so, we add them to results.
                if str(self.ratings[i]['from']) in all_ids:
                    if (self.ratings[i]['time'] >= since and self.ratings[i]['time'] <= until):
                        results.append(self.ratings[i])
                ### Similar with "to" id.        
                if self.ratings[i]['to'] in all_ids:
                    if (self.ratings[i]['time'] >= since and self.ratings[i]['time'] <= until):
                        results.append(self.ratings[i])                
                i+=1
            logging.debug("network get ratings on date: " + str(times))
            return(0,results)
    ### put_ranks defined in similar way as in Java.
    def put_ranks(self,dt1,mydict):
        i = 0
        ### dt1 is date, mydict is the dictionary with ranks.
        while i<len(mydict):
            ### For each rank, we convert ID to string and then convert the ranks to 0-1 range.
            mydict[i]['id'] = str(mydict[i]['id'])
            mydict[i]['rank'] = mydict[i]['rank']*0.01
            i+=1   
        ### we take myreps out of all reputations currently saved in chosen date.
        if dt1 in self.all_reputations:
            myreps = self.all_reputations[dt1]
        else:
            myreps = {}
        ### Then we save mydict in another object, dict_values.
        dict_values = mydict
        i = 0
        while i<len(dict_values):
            ### then we save id and rank.
            the_id = dict_values[i]['id']
            rank = dict_values[i]['rank']
            ### and we change the rank of the id in myreps with 
            myreps[the_id] = rank
            i+=1
        ### Then we change all reputations variable to all the changes we did.
        self.all_reputations[dt1] = myreps
        ### Also, if it is the latest date, we should take into account of it as the latest reputation.
        if dt1 == max(self.all_reputations.keys()):
            self.reputation = myreps
        
        return(0)

    def get_historical_ranks(self,theid):
        all_dates = self.all_reputations.keys()
        ranks = []
        for k in sorted(all_dates):
            if theid in self.all_reputations[k].keys():
                ranks.append(self.all_reputations[k][theid])
        return(ranks)
    
    def calculate_indrating(self,ids,mydate):   
        correlats = dict()
        for k1 in self.predictive_data.keys():
            k = self.predictive_data[k1]
            thevalues = []
            relevant_ranks = []
            for j in k.keys():
                nr_appearances = 1
                
                relevant_ranks.append(self.non_rounded_rep[j])
                thevalues.append(k[j])

                    
            if len(relevant_ranks)!=0:        
                cors = calculate_distance(relevant_ranks,thevalues)
            else:
                cors = 0
                
            correlats[k1] = cors 
            ### I think all cors values should be first normalized.
        for k1 in self.predictive_data.keys():    
            correlats[k1] = 1 - correlats[k1]
        if len(correlats)==0:
            max_correlats = 1
        else:
            max_correlats = max(correlats.values())
        #for k1 in correlats.keys():
        #    correlats[k1] = correlats[k1]/max_correlats 
        for k1 in self.predictive_data.keys():     

            
            if k1 in self.pred_values.keys():
                self.pred_values[k1] = correlats[k1]
            else:
                self.pred_values[k1] = dict()
                self.pred_values[k1] = correlats[k1]
        
        return(0)
    
    
    
    def __init__(self):
        ### we can also initialie everything.
        self.ratings = {}
        self.reputation = {}
        self.all_reputations = {}
        self.set_parameters(dict())### we need changes parameter and in our case that is an empty dictionary

In [6]:
import time
import datetime

# Reputation Scenario Test Data Generation
def dict_sorted(d):
	first = True
	s = "{"
	for key, value in sorted(d.items(), key=lambda x: x[0]): 
		template = "'{}': {}" if first else ", '{}': {}"
		s += template.format(key, value)
		first = False
	s += "}"
	return s


#TODO use any other Reputation Service here
rs = None
#rs = AigentsAPIReputationService('http://localtest.com:1288/', 'john@doe.org', 'q', 'a', False, 'test', True)
rs = PythonReputationService()
if rs is not None:
    rs.set_parameters({'fullnorm':True,'weighting':True,'logratings':False,'logranks':True})

verbose = False


days = 31
consumers = 0.9
suppliers = 0.1
good_range = [1,500]
bad_range = [501,550]

good_transactions = 1
bad_transactions = 2


# Comparing different reputation systems (RS) for different scam periods (SP)
good_agent = {"range": good_range, "values": [100,1000], "transactions": good_transactions, "suppliers": suppliers, "consumers": consumers}
bad_agent = {"range": bad_range, "values": [100,1000], "transactions": bad_transactions, "suppliers": suppliers, "consumers": consumers}
print('Good Agent:',str(good_agent))
print('Bad Agent :',str(bad_agent))
for sp in [10,6,4,2]:

	print('Scam period:',str(sp))
	sip = sp/2

	print('No RS:', end =" ")
	reputation_simulate(good_agent,bad_agent, datetime.date(2018, 1, 1), days, True, None, campaign = [sp,sip], verbose=verbose)

	print('Regular RS:', end =" ")
	rs.set_parameters({'fullnorm':True,'weighting':False,'logratings':False,'denomination':False,'unrated':False,'default':0.5,'decayed':0.5,'ratings':1.0,'spendings':0.0})
	reputation_simulate(good_agent,bad_agent, datetime.date(2018, 1, 1), days, True, rs, campaign = [sp,sip], verbose=verbose)
	
	print('Weighted RS:', end =" ")
	rs.set_parameters({'fullnorm':True,'weighting':True ,'logratings':False,'denomination':True ,'unrated':False,'default':0.5,'decayed':0.5,'ratings':1.0,'spendings':0.0})
	reputation_simulate(good_agent,bad_agent, datetime.date(2018, 1, 1), days, True, rs, campaign = [sp,sip], verbose=verbose)

	print('TOM-based RS:', end =" ")
	rs.set_parameters({'fullnorm':True,'weighting':True ,'logratings':False,'denomination':True ,'unrated':True ,'default':0.0,'decayed':0.5,'ratings':1.0,'spendings':0.0})
	reputation_simulate(good_agent,bad_agent, datetime.date(2018, 1, 1), days, True, rs, campaign = [sp,sip], verbose=verbose)
	
	print('SOM-based RS:', end =" ")
	rs.set_parameters({'fullnorm':True,'weighting':True ,'logratings':False,'denomination':True ,'unrated':False,'default':0.0,'decayed':0.5,'ratings':0.5,'spendings':0.5})
	reputation_simulate(good_agent,bad_agent, datetime.date(2018, 1, 1), days, True, rs, campaign = [sp,sip], verbose=verbose)

del rs

Good Agent: {'range': [1, 500], 'values': [100, 1000], 'transactions': 1, 'suppliers': 0.1, 'consumers': 0.9}
Bad Agent : {'range': [501, 550], 'values': [100, 1000], 'transactions': 2, 'suppliers': 0.1, 'consumers': 0.9}
Scam period: 10
No RS: Good: 7679843 Bad: 744314 Good to Bad: 645420 Good/Bad: 10.318014977549797 Bad/Good_to_Bad: 1.1532242570729139 LTS: 0.0840407805211643 PFS: 0.8671340321423485
Regular RS: Good: 7654005 Bad: 735863 Good to Bad: 344908 Good/Bad: 10.401399445277178 Bad/Good_to_Bad: 2.13350516659515 LTS: 0.0450624215688388 PFS: 0.4687122467089662
Weighted RS: Good: 7661195 Bad: 756323 Good to Bad: 335741 Good/Bad: 10.129527992669798 Bad/Good_to_Bad: 2.252697764050265 LTS: 0.04382358104708208 PFS: 0.4439121909554516
TOM-based RS: Good: 7666241 Bad: 731809 Good to Bad: 161576 Good/Bad: 10.475740254629281 Bad/Good_to_Bad: 4.529193692132495 LTS: 0.02107630062764789 PFS: 0.22078985090371941
SOM-based RS: Good: 7656547 Bad: 732044 Good to Bad: 187405 Good/Bad: 10.45913497

In [7]:
import time
import datetime

# Reputation Scenario Test Data Generation
def dict_sorted(d):
	first = True
	s = "{"
	for key, value in sorted(d.items(), key=lambda x: x[0]): 
		template = "'{}': {}" if first else ", '{}': {}"
		s += template.format(key, value)
		first = False
	s += "}"
	return s


#TODO use any other Reputation Service here
rs = None
#rs = AigentsAPIReputationService('http://localtest.com:1288/', 'john@doe.org', 'q', 'a', False, 'test', True)
rs = PythonReputationService()
if rs is not None:
    rs.set_parameters({'fullnorm':True,'weighting':True,'logratings':False,'logranks':True})

verbose = False


days = 15
consumers = 0.9
suppliers = 0.1
good_range = [1,250]
bad_range = [250,300]

good_transactions = 1
bad_transactions = 2


# Comparing different reputation systems (RS) for different scam periods (SP)
good_agent = {"range": good_range, "values": [100,1000], "transactions": good_transactions, "suppliers": suppliers, "consumers": consumers}
bad_agent = {"range": bad_range, "values": [100,1000], "transactions": bad_transactions, "suppliers": suppliers, "consumers": consumers}
print('Good Agent:',str(good_agent))
print('Bad Agent :',str(bad_agent))
for sp in [10,6,4,2]:

	print('Scam period:',str(sp))
	sip = sp/2

	print('No RS:', end =" ")
	reputation_simulate(good_agent,bad_agent, datetime.date(2018, 1, 1), days, True, None, campaign = [sp,sip], verbose=verbose)

	print('Regular RS:', end =" ")
	rs.set_parameters({'fullnorm':True,'weighting':False,'logratings':False,'denomination':False,'unrated':False,'default':0.5,'decayed':0.5,'ratings':1.0,'spendings':0.0})
	reputation_simulate(good_agent,bad_agent, datetime.date(2018, 1, 1), days, True, rs, campaign = [sp,sip], verbose=verbose)
	
	print('Weighted RS:', end =" ")
	rs.set_parameters({'fullnorm':True,'weighting':True ,'logratings':False,'denomination':True ,'unrated':False,'default':0.5,'decayed':0.5,'ratings':1.0,'spendings':0.0})
	reputation_simulate(good_agent,bad_agent, datetime.date(2018, 1, 1), days, True, rs, campaign = [sp,sip], verbose=verbose)

	print('TOM-based RS:', end =" ")
	rs.set_parameters({'fullnorm':True,'weighting':True ,'logratings':False,'denomination':True ,'unrated':True ,'default':0.0,'decayed':0.5,'ratings':1.0,'spendings':0.0})
	reputation_simulate(good_agent,bad_agent, datetime.date(2018, 1, 1), days, True, rs, campaign = [sp,sip], verbose=verbose)
	
	print('SOM-based RS:', end =" ")
	rs.set_parameters({'fullnorm':True,'weighting':True ,'logratings':False,'denomination':True ,'unrated':False,'default':0.0,'decayed':0.5,'ratings':0.5,'spendings':0.5})
	reputation_simulate(good_agent,bad_agent, datetime.date(2018, 1, 1), days, True, rs, campaign = [sp,sip], verbose=verbose)

del rs

Good Agent: {'range': [1, 250], 'values': [100, 1000], 'transactions': 1, 'suppliers': 0.1, 'consumers': 0.9}
Bad Agent : {'range': [250, 300], 'values': [100, 1000], 'transactions': 2, 'suppliers': 0.1, 'consumers': 0.9}
Scam period: 10
No RS: Good: 1869811 Bad: 254171 Good to Bad: 286151 Good/Bad: 7.356508020191131 Bad/Good_to_Bad: 0.8882408239006678 LTS: 0.15303739254930043 PFS: 1.1258208056780672
Regular RS: Good: 1856184 Bad: 259365 Good to Bad: 127830 Good/Bad: 7.1566479671505405 Bad/Good_to_Bad: 2.028983806618165 LTS: 0.06886709507247127 PFS: 0.49285755595396447
Weighted RS: Good: 1865726 Bad: 257625 Good to Bad: 129421 Good/Bad: 7.242022319262494 Bad/Good_to_Bad: 1.9905965801531436 LTS: 0.06936763490458941 PFS: 0.5023619602134886
TOM-based RS: Good: 1849405 Bad: 255351 Good to Bad: 96766 Good/Bad: 7.242599402391218 Bad/Good_to_Bad: 2.6388504226691194 LTS: 0.05232277408139375 PFS: 0.37895289229335305
SOM-based RS: Good: 1865926 Bad: 254165 Good to Bad: 210875 Good/Bad: 7.3413963

In [8]:
import time
import datetime

# Reputation Scenario Test Data Generation
def dict_sorted(d):
	first = True
	s = "{"
	for key, value in sorted(d.items(), key=lambda x: x[0]): 
		template = "'{}': {}" if first else ", '{}': {}"
		s += template.format(key, value)
		first = False
	s += "}"
	return s


#TODO use any other Reputation Service here
rs = None
#rs = AigentsAPIReputationService('http://localtest.com:1288/', 'john@doe.org', 'q', 'a', False, 'test', True)
rs = PythonReputationService()
if rs is not None:
    rs.set_parameters({'fullnorm':True,'weighting':True,'logratings':False,'logranks':True})

verbose = False


days = 31
consumers = 0.9
suppliers = 0.1
good_range = [1,500]
bad_range = [501,550]

good_transactions = 1
bad_transactions = 2


# Comparing different reputation systems (RS) for different scam periods (SP)
good_agent = {"range": good_range, "values": [100,1000], "transactions": good_transactions, "suppliers": suppliers, "consumers": consumers}
bad_agent = {"range": bad_range, "values": [100,1000], "transactions": bad_transactions, "suppliers": suppliers, "consumers": consumers}
print('Good Agent:',str(good_agent))
print('Bad Agent :',str(bad_agent))
for sp in [20,15,12,8]:

	print('Scam period:',str(sp))
	sip = sp/2

	print('No RS:', end =" ")
	reputation_simulate(good_agent,bad_agent, datetime.date(2018, 1, 1), days, True, None, campaign = [sp,sip], verbose=verbose)

	print('Regular RS:', end =" ")
	rs.set_parameters({'fullnorm':True,'weighting':False,'logratings':False,'denomination':False,'unrated':False,'default':0.5,'decayed':0.5,'ratings':1.0,'spendings':0.0})
	reputation_simulate(good_agent,bad_agent, datetime.date(2018, 1, 1), days, True, rs, campaign = [sp,sip], verbose=verbose)
	
	print('Weighted RS:', end =" ")
	rs.set_parameters({'fullnorm':True,'weighting':True ,'logratings':False,'denomination':True ,'unrated':False,'default':0.5,'decayed':0.5,'ratings':1.0,'spendings':0.0})
	reputation_simulate(good_agent,bad_agent, datetime.date(2018, 1, 1), days, True, rs, campaign = [sp,sip], verbose=verbose)

	print('TOM-based RS:', end =" ")
	rs.set_parameters({'fullnorm':True,'weighting':True ,'logratings':False,'denomination':True ,'unrated':True ,'default':0.0,'decayed':0.5,'ratings':1.0,'spendings':0.0})
	reputation_simulate(good_agent,bad_agent, datetime.date(2018, 1, 1), days, True, rs, campaign = [sp,sip], verbose=verbose)
	
	print('SOM-based RS:', end =" ")
	rs.set_parameters({'fullnorm':True,'weighting':True ,'logratings':False,'denomination':True ,'unrated':False,'default':0.0,'decayed':0.5,'ratings':0.5,'spendings':0.5})
	reputation_simulate(good_agent,bad_agent, datetime.date(2018, 1, 1), days, True, rs, campaign = [sp,sip], verbose=verbose)

del rs

Good Agent: {'range': [1, 500], 'values': [100, 1000], 'transactions': 1, 'suppliers': 0.1, 'consumers': 0.9}
Bad Agent : {'range': [501, 550], 'values': [100, 1000], 'transactions': 2, 'suppliers': 0.1, 'consumers': 0.9}
Scam period: 20
No RS: Good: 7684382 Bad: 538529 Good to Bad: 611270 Good/Bad: 14.269207415013861 Bad/Good_to_Bad: 0.881000212671978 LTS: 0.07954706051833446 PFS: 1.1350735057907746
Regular RS: Good: 7653715 Bad: 546332 Good to Bad: 277074 Good/Bad: 14.009274580291837 Bad/Good_to_Bad: 1.9717909294989786 LTS: 0.03620124344844301 PFS: 0.5071531596172291
Weighted RS: Good: 7649774 Bad: 545798 Good to Bad: 278984 Good/Bad: 14.0157604095288 Bad/Good_to_Bad: 1.956377426662461 LTS: 0.03646957413382408 PFS: 0.5111488132972272
TOM-based RS: Good: 7661687 Bad: 549034 Good to Bad: 255151 Good/Bad: 13.954849790723344 Bad/Good_to_Bad: 2.151800306485179 LTS: 0.033302195717470576 PFS: 0.4647271389385721
SOM-based RS: Good: 7651157 Bad: 546474 Good to Bad: 292504 Good/Bad: 14.0009533

In [9]:
import time
import datetime

# Reputation Scenario Test Data Generation
def dict_sorted(d):
	first = True
	s = "{"
	for key, value in sorted(d.items(), key=lambda x: x[0]): 
		template = "'{}': {}" if first else ", '{}': {}"
		s += template.format(key, value)
		first = False
	s += "}"
	return s


#TODO use any other Reputation Service here
rs = None
#rs = AigentsAPIReputationService('http://localtest.com:1288/', 'john@doe.org', 'q', 'a', False, 'test', True)
rs = PythonReputationService()
if rs is not None:
    rs.set_parameters({'fullnorm':True,'weighting':True,'logratings':False,'logranks':True})

verbose = False


days = 90
consumers = 0.9
suppliers = 0.1
good_range = [1,750]
bad_range = [751,800]

good_transactions = 1
bad_transactions = 2


# Comparing different reputation systems (RS) for different scam periods (SP)
good_agent = {"range": good_range, "values": [100,1000], "transactions": good_transactions, "suppliers": suppliers, "consumers": consumers}
bad_agent = {"range": bad_range, "values": [100,1000], "transactions": bad_transactions, "suppliers": suppliers, "consumers": consumers}
print('Good Agent:',str(good_agent))
print('Bad Agent :',str(bad_agent))
for sp in [10,6,4,2]:

	print('Scam period:',str(sp))
	sip = sp/2

	print('No RS:', end =" ")
	reputation_simulate(good_agent,bad_agent, datetime.date(2018, 1, 1), days, True, None, campaign = [sp,sip], verbose=verbose)

	print('Regular RS:', end =" ")
	rs.set_parameters({'fullnorm':True,'weighting':False,'logratings':False,'denomination':False,'unrated':False,'default':0.5,'decayed':0.5,'ratings':1.0,'spendings':0.0})
	reputation_simulate(good_agent,bad_agent, datetime.date(2018, 1, 1), days, True, rs, campaign = [sp,sip], verbose=verbose)
	
	print('Weighted RS:', end =" ")
	rs.set_parameters({'fullnorm':True,'weighting':True ,'logratings':False,'denomination':True ,'unrated':False,'default':0.5,'decayed':0.5,'ratings':1.0,'spendings':0.0})
	reputation_simulate(good_agent,bad_agent, datetime.date(2018, 1, 1), days, True, rs, campaign = [sp,sip], verbose=verbose)

	print('TOM-based RS:', end =" ")
	rs.set_parameters({'fullnorm':True,'weighting':True ,'logratings':False,'denomination':True ,'unrated':True ,'default':0.0,'decayed':0.5,'ratings':1.0,'spendings':0.0})
	reputation_simulate(good_agent,bad_agent, datetime.date(2018, 1, 1), days, True, rs, campaign = [sp,sip], verbose=verbose)
	
	print('SOM-based RS:', end =" ")
	rs.set_parameters({'fullnorm':True,'weighting':True ,'logratings':False,'denomination':True ,'unrated':False,'default':0.0,'decayed':0.5,'ratings':0.5,'spendings':0.5})
	reputation_simulate(good_agent,bad_agent, datetime.date(2018, 1, 1), days, True, rs, campaign = [sp,sip], verbose=verbose)

del rs

Good Agent: {'range': [1, 750], 'values': [100, 1000], 'transactions': 1, 'suppliers': 0.1, 'consumers': 0.9}
Bad Agent : {'range': [751, 800], 'values': [100, 1000], 'transactions': 2, 'suppliers': 0.1, 'consumers': 0.9}
Scam period: 10
No RS: Good: 33420499 Bad: 2205323 Good to Bad: 1942642 Good/Bad: 15.154468982548135 Bad/Good_to_Bad: 1.1352184293348955 LTS: 0.0581272589616331 PFS: 0.8808877429746119
Regular RS: Good: 33392315 Bad: 2256211 Good to Bad: 895192 Good/Bad: 14.800173831259576 Bad/Good_to_Bad: 2.5203654634983335 LTS: 0.026808324011078598 PFS: 0.3967678554886932
Weighted RS: Good: 33369692 Bad: 2234046 Good to Bad: 886622 Good/Bad: 14.936886706898605 Bad/Good_to_Bad: 2.519727685530023 LTS: 0.02656967885708984 PFS: 0.39686828292703014
TOM-based RS: Good: 33401093 Bad: 2215890 Good to Bad: 410073 Good/Bad: 15.073443627616895 Bad/Good_to_Bad: 5.403647643224499 LTS: 0.012277232963603916 PFS: 0.18506017898000351
SOM-based RS: Good: 33400472 Bad: 2217178 Good to Bad: 321122 Good